In [2]:
import Ipynb_importer
from a_csp_darknet53  import *
from b_yolov4_neck import *

In [1]:
from functools import wraps

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Add, BatchNormalization, Concatenate,
                                     Conv2D, LeakyReLU, MaxPooling2D,Reshape,
                                     UpSampling2D, ZeroPadding2D)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

from scipy.special import expit, softmax

## 1、解码

在Yolov1中，网络直接回归检测框的宽、高，这样效果有限。所以在Yolov2中，改为了回归基于先验框的变化值，这样网络的学习难度降低，整体精度提升不小。Yolov3沿用了Yolov2中关于先验框的技巧，并且**使用k-means对数据集中的标签框进行聚类**，得到类别中心点的9个框，作为先验框。

在COCO数据集中（原始图片全部resize为416 × 416），九个框分别是 (10×13)，(16×30)，(33×23)，(30×61)，(62×45)，(59× 119)， (116 × 90)， (156 × 198)，(373 × 326) ，顺序为w × h。

> 注：先验框只与检测框的w、h有关，与x、y无关。

1. **检测框解码**

有了先验框与输出特征图，就可以解码检测框 x，y，w，h。

![[公式]](https://www.zhihu.com/equation?tex=b_x%3D%5Csigma+%28t_x%29+%2B+c_x+%5C%5C+b_y%3D%5Csigma+%28t_y%29+%2B+c_y+%5C%5C+b_w%3Dp_we%5E%7Bt_w%7D++%5C%5C+b_h%3Dp_he%5E%7Bt_h%7D+%5C%5C)


这里记特征图的大小为 ![[公式]](https://www.zhihu.com/equation?tex=%28W%2C+H%29) （在文中是 ![[公式]](https://www.zhihu.com/equation?tex=%2813%2C+13%29) )，这样我们可以将边界框相对于整张图片的位置和大小计算出来、

![[公式]](https://www.zhihu.com/equation?tex=%5C%5Cb_x+%3D+%28%5Csigma+%28t_x%29%2Bc_x%29%2FW)

![[公式]](https://www.zhihu.com/equation?tex=%5C%5C+b_y+%3D+%28%5Csigma+%28t_y%29+%2B+c_y%29%2FH)

如下图所示， ![[公式]](https://www.zhihu.com/equation?tex=%5Csigma%28t_x%29%2C+%5Csigma%28t_y%29) 是基于矩形框中心点左上角格点坐标的偏移量， ![[公式]](https://www.zhihu.com/equation?tex=%5Csigma) 是**激活函数**，论文中作者使用**sigmoid**。 ![[公式]](https://www.zhihu.com/equation?tex=p_w%2C+p_h) 是先验框的宽、高，通过上述公式，计算出实际预测框的宽高 ![[公式]](https://www.zhihu.com/equation?tex=%28b_w%2C+b_h%29) 。

<img src="https://pic2.zhimg.com/80/v2-758b1df9132a9f4b4e0c7def735e9a11_1440w.jpg" alt="img" style="zoom:40%;" />

举个具体的例子，假设对于第二个特征图26 × 26 × 3 × 85中的第[5，4，2]维，上图中的 ![[公式]](https://www.zhihu.com/equation?tex=c_y) 为5， ![[公式]](https://www.zhihu.com/equation?tex=+c_x) 为4，第二个特征图对应的先验框为(30×61)，(62×45)，(59× 119)，prior_box的index为2，那么取最后一个59，119作为先验w、先验h。这样计算之后的 ![[公式]](https://www.zhihu.com/equation?tex=b_x%2Cb_y) 还需要乘以特征图二的采样率16，得到真实的检测框x，y。

2. **检测置信度解码**

物体的检测置信度，在Yolo设计中非常重要，关系到算法的检测正确率与召回率。

置信度在输出85维中占固定一位，由sigmoid函数解码即可，解码之后数值区间在[0，1]中。

3. **类别解码**

   > https://zhuanlan.zhihu.com/p/42865896
   >
   > 物体之间的相互覆盖都是不能避免的。因此一个锚点的感受野肯定会包含两个甚至更多个不同物体的可能。如果使用softmax作为激活函数，意味着在一个锚点中的检测是互斥的，只有一个或者说少数点的置信度可以大于阈值。使用sigmoid分类器，最终各类别之间的互斥被取消。

COCO数据集有80个类别，所以类别数在85维输出中占了80维，每一维独立代表一个类别的置信度。使用sigmoid激活函数替代了Yolov2中的softmax，**取消了类别之间的互斥，可以使网络更加灵活。**

三个特征图一共可以解码出 13 × 13 × 3 + 26 × 26 × 3 + 52 × 52 × 3 = 10647 个box以及相应的类别、置信度。这10647个box，在训练和推理时，使用方法不一样：

1. 训练时10647个box全部送入打标签函数，进行后一步的标签以及损失函数的计算。
2. 推理时，选取一个置信度阈值，过滤掉低阈值box，再经过nms（非极大值抑制），就可以输出整个网络的预测结果了。

In [ ]:
def yolo_decode(feats, anchors, num_classes, input_shape, scale_x_y=None, calc_loss=False):
    """Decode final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    # ----------------------------------------------------------------------------------------------------------
    # 生成 grid 网格基准 (13, 13, 1, 2)
    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    # Reshape to ([batch_size, height, width, num_anchors, (num_classes+5)])
    feats = K.reshape(feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    # box_xy 数值范围调整为【0-1】，1为grid_shape高或宽的长度
    #box_wh 数值范围调整为 【0-1】，1为input_shape大小，输入尺寸是使用backbone的最小特征图尺寸*stride得到的
    # 强调说明一下：这里 box_xy 是相对于grid 的位置（说成input似乎也行）；box_wh是相对于 input_shape大小
    # scale_x_y是一个 trick，见下文链接
    if scale_x_y:
        # Eliminate grid sensitivity trick involved in YOLOv4
        #
        # Reference Paper & code:
        #     "YOLOv4: Optimal Speed and Accuracy of Object Detection"
        #     https://arxiv.org/abs/2004.10934
        #     https://github.com/opencv/opencv/issues/17148
        #     https://zhuanlan.zhihu.com/p/139724869
        box_xy_tmp = K.sigmoid(feats[..., :2]) * scale_x_y - (scale_x_y - 1) / 2
        box_xy = (box_xy_tmp + grid) / K.cast(grid_shape[..., ::-1], K.dtype(feats))
    else:
        box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[..., ::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[..., ::-1], K.dtype(feats))
    # sigmoid objectness scores 置信度解码
    box_confidence = K.sigmoid(feats[..., 4:5])
    # class prons 类别解码
    box_class_probs = K.sigmoid(feats[..., 5:])

    #   在计算loss的时候返回grid, feats, box_xy, box_wh
    #   在预测的时候返回box_xy, box_wh, box_confidence, box_class_probs
    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs

## 2、计算 boxes 真实宽高和位置

输入尺寸 input_size 为 416\*416，图片尺寸 image_size 为 需要按照纵横比例缩放到 416\*416， **取 min(w/img_w, h/img_h)这个比例来缩放，保证长的边缩放为需要的输入尺寸416，而短边按比例缩放不会扭曲**。

例如：image_shape是（768,576）, 缩放后的尺寸为new_w, new_h=416,312，即new_shape为（416，312）

需要的输入尺寸是w,h=416\*416.如图2所示：

<img src="https://pic4.zhimg.com/80/v2-62f64986f2a5bf499045e274bcbc782b_720w.jpg" alt="img" style="zoom:37%;" />

现在已经得到相对于 grid（或者说input_shape）的位置和宽高

需要转化成相对于 iamge_shape 的位置和宽高

In [ ]:
def yolo_corrected_boxes(box_xy, box_wh, input_shape, image_shape):
    """计算出预测框相对于原图像的位置和大小"""
    input_shape    = K.cast(input_shape, K.dtype(box_xy))
    image_shape  = K.cast(image_shape, K.dtype(box_xy))
    
    # reshape the image_shape tensor to align with boxes dimension
    image_shape = K.reshape(image_shape, [-1, 1, 1, 1, 2])
    
    new_shape = K.round(image * K.min(input_shape/image_shape))  # （416，312）
    
    #  这里求出来的offset是图像有效区域相对于图像左上角的偏移情况
    offset = (input_shape-new_shape)/2./input_shape  #（0，0.125）
    scale = input_shape/new_shape  # (1, 1.333)
    
    # 举例，在现在还未进行任何尺度变换，假设有一个坐标为（0.4，0.3），wh为（0.2，0.2）的框。
    # 上面这个数值的意义为：这个坐标是在input（416，416） 的相对位置和相对宽高，实际上，input
    # 的（:,0:0.125）的区域都没有意义，（:,0.875:1）也是。
    # 
    # 现在我们需要把上面这个相对参照物换成 image，而image经过缩放后的区域是（416，312）。上面的
    # offset 和 Scale 就是为了完成一个参照物替换的
    box_xy_image = (box_xy - offset) * scale
    box_wh_image = box_wh * scale
    # box_xy_image 和 box_wh_image 就是以image 为参考系的相对位置和相对宽高
    
    box_mins = box_xy_image - (box_wh_image / 2.)
    box_maxes = box_xy_image + (box_wh_image / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # x_min
        box_mins[..., 1:2],  # y_min
        box_maxes[..., 0:1],  # x_max
        box_maxes[..., 1:2]  # y_max
    ])
    
    # Scale boxes back to original image shape.
    # 通过乘积获得了真实宽高和位置了
    image_wh = image_shape[..., ::-1]
    boxes *= K.concatenate([image_wh, image_wh])
    return boxes

## 3、outputs处理函数
处理 yolo_body 输出的特征图，获得 bounding boxes 和 scores

此时可以获得所有框的位置和scores了

In [ ]:
def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape, scale_x_y):
    """Process Conv layer output"""
    box_xy, box_wh, box_confidence, box_class_probs = yolo3_decode(feats,
        anchors, num_classes, input_shape, scale_x_y=scale_x_y)
    
    num_anchors = len(anchors)
    grid_shape = K.shape(feats)[1:3] # height, width
    total_anchor_num = grid_shape[0] * grid_shape[1] * num_anchors
    
    boxes = yolo3_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, total_anchor_num, 4])
    
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, total_anchor_num, num_classes])
    
    return boxes, box_scores

In [2]:
def yolov4_head(
    input_shapes,
    anchors,
    num_classes,
    training,
    yolo_max_boxes,
    yolo_iou_threshold,
    yolo_score_threshold,
):
    """
    Args:
        input_shapes (List[Tuple[int]]): List of 3 tuples, which are the output shapes of the neck.
            None dimensions are ignored.
            For CSPDarknet53+YOLOv4_neck, those are: [ (52, 52, 128), (26, 26, 256), (13, 13, 512)] for a (416,
            416) input.
        anchors (List[numpy.array[int, 2]]): List of 3 numpy arrays containing the anchor sizes used for each stage.
            The first and second columns of the numpy arrays respectively contain the anchors width and height.
        num_classes (int): Number of classes.
        training (boolean): If False, will output boxes computed through YOLO regression and NMS, and YOLO features
            otherwise. Set it True for training, and False for inferences.
        yolo_max_boxes (int): Maximum number of boxes predicted on each image (across all anchors/stages)
        yolo_iou_threshold (float between 0. and 1.): IOU threshold defining whether close boxes will be merged
            during non max regression.
        yolo_score_threshold (float between 0. and 1.): Boxes with score lower than this threshold will be filtered
            out during non max regression.
    Returns:
        tf.keras.Model: Head model
    """
    input_1 = tf.keras.Input(shape=filter(None, input_shapes[0]))  # 52* 52* 128
    input_2 = tf.keras.Input(shape=filter(None, input_shapes[1]))  # 26* 26* 256
    input_3 = tf.keras.Input(shape=filter(None, input_shapes[2]))  # 13* 13* 512
    
    # p3 输出
    P3_out = darknet_CBL(256, (3,3))(input_1)
    output_1 = DarknetConv2D(len(anchors[0])*(num_classes+5),(1,1))(P3_out)  # len(anchor[0]) = 3, num_classes=80，整个网络输入为(416, 416)的情况下，此时的输出为13* 13* (3*85)
    output_1 = Reshape( (P3.shape[1], P3.shape[2], len(anchors[0]), num_classes + 5))(output_1)  # 13* 13* 3* 85
    
    # p3 下采样与 p4 实现 FPN，获得 concatenate 后的 p4
    P3_downsample = darknet_CBL(256, (3,3), strides=(2,2))(input_1)
    P4 = Concatenate()([P3_downsample, input_2])
    # p4 CBL*5
    P4 = make_five_convs(P4,256)
    # p4 输出
    P4_out = darknet_CBL(512, (3,3))(P4)
    output_2 = DarknetConv2D(len(anchors[1])*(num_classes+5), (1,1))(P4_out)
    output_2 = Reshape( (P4.shape[1], P4.shape[2], len(anchors[1]), num_classes + 5))(output_2)  # 26* 26* 3* 85
    
    
    # p4 下采样与 p5 实现 FPN，获得 concatenate 后的 p5
    P4_downsample = darknet_CBL(512, (3,3), strides=(2,2))(P4)
    P5 = Concatenate()([P4_downsample, input_3])
    # p5 CBL*5
    P5 = make_five_convs(P5,512)
    # p5输出
    P5_out = darknet_CBL(1024, (3,3))(P5)
    output_3 = DarknetConv2D(len(anchors[2])*(num_classes+5), (1,1))(P5_out)
    output_3 = Reshape( (P5.shape[1], P5.shape[2], len(anchors[2]), num_classes + 5))(output_3)  # 52* 52* 3* 85

    # 三张特征图 output_1(13* 13* 3* 85)、output_2(26* 26* 3* 85)、output_3(52* 52* 3* 85）从上往下
    # 是整个 yolo 输出的检测结果
    # 检测框位置（4维）、检测置信度（1维）、类别（80维）都在其中，加起来正好是85维。
    # 特征图其他维度N × N × 3，N × N代表了检测框的参考位置信息，3是3个不同尺度的先验框
    # 训练阶段则直接返回特征图结果，推理阶段则解码检测信息
    
    # 训练阶段
    if training:
        return tf.keras.Model(
            [input_1, input_2, input_3],
            [output_1, output_2, output_3],
            name="YOLOv3_head",
        )
    
    # 推理阶段
    # 解码三张特征图的信息
    predictions_1 = tf.keras.layers.Lambda(
        lambda x_input: yolov3_boxes_regression(x_input, anchors[0]),
        name="yolov3_boxes_regression_small_scale",
    )(output_1)
    predictions_2 = tf.keras.layers.Lambda(
        lambda x_input: yolov3_boxes_regression(x_input, anchors[1]),
        name="yolov3_boxes_regression_medium_scale",
    )(output_2)
    predictions_3 = tf.keras.layers.Lambda(
        lambda x_input: yolov3_boxes_regression(x_input, anchors[2]),
        name="yolov3_boxes_regression_large_scale",
    )(output_3)
    
    # nms处理
    output = tf.keras.layers.Lambda(
        lambda x_input: yolo_nms(
            x_input,
            yolo_max_boxes=yolo_max_boxes,
            yolo_iou_threshold=yolo_iou_threshold,
            yolo_score_threshold=yolo_score_threshold,
        ),
        name="yolov4_nms",
    )([predictions_1, predictions_2, predictions_3])

    return tf.keras.Model([input_1, input_2, input_3], output, name="YOLOv3_head")
    